# 1. Import Dependencies

In [4]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
from ale_py import ALEInterface
import os

# 2. Test Environment

In [22]:
environment_name = "ALE/Breakout-v5"
env = gym.make(environment_name, render_mode="rgb_array")

In [9]:
env.reset()

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], shape=(210, 160, 3), dtype=uint8),
 {'lives': 5, 'episode_frame_number': 0, 'frame_number': 0})

In [11]:
env.action_space

Discrete(4)

In [12]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [23]:
episode =5
for episode in range(episode+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward
    print(f"Episode: {episode} Score: {score}")
env.close()

Episode: 0 Score: 6.0
Episode: 1 Score: 2.0
Episode: 2 Score: 3.0
Episode: 3 Score: 0.0
Episode: 4 Score: 3.0
Episode: 5 Score: 2.0


# 3. Vectorise Environment and Train Model

In [24]:
env = make_atari_env(environment_name, n_envs=4, seed=42)
env = VecFrameStack(env, n_stack=4)

MLP is good for tabular data and CNN for image data

In [27]:
log_path = os.path.join('Training', 'Logs')
model = A2C("CnnPolicy", env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [28]:
model.learn(total_timesteps=100000)

Logging to Training/Logs/A2C_1
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 236      |
|    ep_rew_mean        | 1.87     |
| time/                 |          |
|    fps                | 113      |
|    iterations         | 100      |
|    time_elapsed       | 17       |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.36    |
|    explained_variance | -0.0144  |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.0856   |
|    value_loss         | 0.174    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 241      |
|    ep_rew_mean        | 1.94     |
| time/                 |          |
|    fps                | 123      |
|    iterations         | 200      |
|    time_elapsed       | 32       |
|    total_timesteps    | 4000     |
| train

# 4. Save and Reload Model

In [29]:
a2c_model = os.path.join('Training', 'Saved_Models', 'A2C_Breakout_Model')
model.save(a2c_model)

In [43]:
del model

In [45]:
env = make_atari_env(environment_name, n_envs=1, seed=42, env_kwargs={"render_mode": "human"})
env = VecFrameStack(env, n_stack=4)

In [46]:
model = A2C.load(a2c_model, env)

Wrapping the env in a VecTransposeImage.


# 5. Evaluate and Test

In [47]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(np.float64(13.3), np.float64(3.4655446902326914))

In [48]:
env.close()